In [5]:
!pip install opencv-python torch torchvision matplotlib facenet-pytorch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of facenet-pytorch to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/2

In [2]:
import os
import cv2
import time
import torch
import copy
import numpy as np
from PIL import Image
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Face Dataset Class**

In [4]:
from facenet_pytorch import MTCNN

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class FaceDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.data = []
        self.labels = []
        self.transform = transform
        self.class_names = sorted([d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))])


        self.mtcnn = MTCNN(keep_all=False, min_face_size=40, device=device)

        print("Loading dataset and detecting faces...")
        for label, person in enumerate(self.class_names):
            person_dir = os.path.join(root_dir, person)
            for img_name in os.listdir(person_dir):
                img_path = os.path.join(person_dir, img_name)
                try:
                    img = Image.open(img_path).convert('RGB')

                    face_tensor = self.mtcnn(img)

                    if face_tensor is not None:
                        face = transforms.ToPILImage()(face_tensor)
                        self.data.append(face)
                        self.labels.append(label)
                except Exception as e:
                    print(f"Could not process image {img_path}: {e}")
        print("Dataset loaded successfully.")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = self.data[idx]
        if self.transform:
            image = self.transform(image)
        return image, self.labels[idx]

# **Load Dataset from Drive**

In [5]:
dataset_path = "/content/drive/My Drive/faces_dataset"

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

full_dataset = FaceDataset(root_dir=dataset_path, transform=transform)

train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

print(f"Total classes: {len(full_dataset.class_names)}")
print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")

Loading dataset and detecting faces...
Dataset loaded successfully.
Total classes: 5
Training set size: 1540
Validation set size: 385


# **Setup VGG16 Model**

In [6]:
model = models.vgg16(weights=models.VGG16_Weights.DEFAULT)

for param in model.parameters():
    param.requires_grad = False

for param in model.features[24:].parameters():
    param.requires_grad = True

for param in model.classifier.parameters():
    param.requires_grad = True

num_classes = len(full_dataset.class_names)
model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)

model = model.to(device)

for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:03<00:00, 153MB/s]


features.24.weight
features.24.bias
features.26.weight
features.26.bias
features.28.weight
features.28.bias
classifier.0.weight
classifier.0.bias
classifier.3.weight
classifier.3.bias
classifier.6.weight
classifier.6.bias


# **Train the Model**

In [7]:
patience = 7
best_val_loss = float('inf')
counter = 0

optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001, momentum=0.9, nesterov=True)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.1, verbose=True)

criterion = nn.CrossEntropyLoss()

best_model_wts = copy.deepcopy(model.state_dict())

print("\nStarting model training...")
for epoch in range(50):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct_train += torch.sum(preds == labels.data)
        total_train += inputs.size(0)

    epoch_loss = running_loss / total_train
    epoch_acc = correct_train.double() / total_train

    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct_val += torch.sum(preds == labels.data)
            total_val += inputs.size(0)

    epoch_val_loss = val_loss / total_val
    epoch_val_acc = correct_val.double() / total_val

    print(f"Epoch {epoch+1:02d} | Train Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f} | Val Loss: {epoch_val_loss:.4f}, Val Acc: {epoch_val_acc:.4f}")

    scheduler.step(epoch_val_loss)

    if epoch_val_loss < best_val_loss:
        print(f"Validation loss decreased ({best_val_loss:.4f} --> {epoch_val_loss:.4f}). Saving model...")
        best_val_loss = epoch_val_loss
        best_model_wts = copy.deepcopy(model.state_dict())
        counter = 0
    else:
        counter += 1
        print(f"No improvement for {counter} epoch(s)")
        if counter >= patience:
            print("Early stopping triggered")
            break

model.load_state_dict(best_model_wts)
model_save_path = "/content/drive/My Drive/models/face_recognition_vgg16_best.pth"
torch.save(model.state_dict(), model_save_path)
print(f"Best model saved to: {model_save_path}")

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(



Starting model training...
Epoch 01 | Train Loss: 0.2686, Acc: 0.9032 | Val Loss: 0.0081, Val Acc: 0.9974
Validation loss decreased (inf --> 0.0081). Saving model...
Epoch 02 | Train Loss: 0.0397, Acc: 0.9877 | Val Loss: 0.0069, Val Acc: 0.9974
Validation loss decreased (0.0081 --> 0.0069). Saving model...
Epoch 03 | Train Loss: 0.0220, Acc: 0.9935 | Val Loss: 0.0203, Val Acc: 0.9948
No improvement for 1 epoch(s)
Epoch 04 | Train Loss: 0.0029, Acc: 0.9994 | Val Loss: 0.0028, Val Acc: 0.9974
Validation loss decreased (0.0069 --> 0.0028). Saving model...
Epoch 05 | Train Loss: 0.0011, Acc: 1.0000 | Val Loss: 0.0055, Val Acc: 0.9974
No improvement for 1 epoch(s)
Epoch 06 | Train Loss: 0.0027, Acc: 0.9994 | Val Loss: 0.0004, Val Acc: 1.0000
Validation loss decreased (0.0028 --> 0.0004). Saving model...
Epoch 07 | Train Loss: 0.0050, Acc: 0.9981 | Val Loss: 0.0004, Val Acc: 1.0000
Validation loss decreased (0.0004 --> 0.0004). Saving model...
Epoch 08 | Train Loss: 0.0007, Acc: 1.0000 | Va